# Capstone Project 2 - Part 2

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from pylab import rcParams
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pickle
# fix the warning problem in jupyter notebook
import warnings
warnings.filterwarnings('ignore')

In [2]:
# read pickle file into python
infile = open('C:\A\Education\Data Science\Capstone project 2\dataframe','rb')
df= pickle.load(infile)
infile.close()

In [3]:
df.head()

,order_count
order_purchase_timestamp_day,
2017-03-31,70
2017-04-01,68
2017-04-02,65
2017-04-03,72
2017-04-04,96


In [27]:
# define tune_model function for tuning the hypeparameter Q, train the SARIMAX model with 
# first 80% of the data and perform one step ahead forecast for the rest of 20%. For example: 
# first df_train is 0~397(n=0), second dr_train is 0~398(n=1),etc.
# For the SARIMAX model, P=7 according to above analysis, D=2 (2 degree function based on 
# excel search, S=7.
# use Symmetric Mean Absolute Percentage Error (SMAPE) to measure the performance of the model
def tune_model(q,n):
    # create empty list for error
    error_list=[]
    #iterate n times to get n predicted values
    for i in range(n):
        # split training dataset
        df_train=df.iloc[:398+i]
        # instantiate SARIMAX model, P=7, D=2, S=7 
        model=SARIMAX(df_train,seasonal_order=(7,2,q,7),enforce_stationarity=False,enforce_invertibility=False)
        # train the model
        model_fit=model.fit()
        # get one step ahead forecast
        model_fore=model_fit.forecast()
        #convert model_fit (series object) to array 
        fore=model_fore.array
        # get the predicted value from array
        fore_val=fore[0]
        # get the actual value
        actual_val=df.order_count.iloc[398+i]
        # calculate |Ft−At|/(Ft+At) 
        error=abs(fore_val-actual_val)/(fore_val+actual_val)
        error_list.append(error)
    # calculate smape
    smape=sum(error_list)/n
    return smape

In [30]:
# research for Q 
Q_list=[]
smape_list=[]
for k in range(8):
    result=tune_model(k,100)
    Q_list.append(k)
    smape_list.append(result)

In [32]:
df_tune = pd.DataFrame()
df_tune['Q']=Q_list
df_tune['smape']=smape_list

In [33]:
# as the results showed below, when Q=4, smape has the lowest score 0.0667
print(df_tune)

   Q     smape
0  0  0.078231
1  1  0.067803
2  2  0.067559
3  3  0.072270
4  4  0.066720
5  5  0.069321
6  6  0.071007
7  7  0.068680


In [35]:
# define tune_model function for tuning the hypeparameter Q, train the SARIMAX model with 
# first 80% of the data and perform one step ahead forecast for the rest of 20%. For example: 
# first df_train is 0~397(n=0), second dr_train is 0~398(n=1),etc.
# For the SARIMAX model, P=7 according to above analysis, D=1 (1 degree function based on 
# excel search, S=7.
# use Symmetric Mean Absolute Percentage Error (SMAPE) to measure the performance of the model
def tune_model_1(q,n):
    # create empty list for error
    error_list=[]
    #iterate n times to get n predicted values
    for i in range(n):
        # split training dataset
        df_train=df.iloc[:398+i]
        # instantiate SARIMAX model, P=7, D=1, Q=1, S=7 
        model=SARIMAX(df_train,seasonal_order=(7,1,q,7),enforce_stationarity=False,enforce_invertibility=False)
        # train the model
        model_fit=model.fit()
        # get one step ahead forecast
        model_fore=model_fit.forecast()
        #convert model_fit (series object) to array 
        fore=model_fore.array
        # get the predicted value from array
        fore_val=fore[0]
        # get the actual value
        actual_val=df.order_count.iloc[398+i]
        # calculate |Ft−At|/(Ft+At) 
        error=abs(fore_val-actual_val)/(fore_val+actual_val)
        error_list.append(error)
    # calculate smape
    smape=sum(error_list)/n
    return smape

In [ ]:
Q_list=[]
smape_list=[]
for k in range(8):
    result=tune_model_1(k,100)
    Q_list.append(k)
    smape_list.append(result)

In [16]:
df_tune = pd.DataFrame()
df_tune['Q']=Q_list
df_tune['smape']=smape_list

In [17]:
print(df_tune)

   Q     smape
0  0  0.066236
1  1  0.066236
2  2  0.066236
3  3  0.066236
4  4  0.066236
5  5  0.066236
6  6  0.066236
7  7  0.066236
